In [9]:
import sqlite3
import pandas as pd
import json
from progressbar import ProgressBar
from sklearn.utils import shuffle
pd.set_option('display.max_rows', 500)

In [10]:
path = './fastai-project/homework/data/NELA19/'
labels_data = pd.read_csv(path + 'labels.csv')

In [11]:
with open(path + 'source-metadata.json') as file:
    metadata = (json.loads(file.read()))
    
for source in metadata.keys():
    labels_data.loc[labels_data.source == source, 'n_articles'] = metadata[source]['articles']
    
cols = ['source', 'n_articles', 'aggregated_label', 'Media Bias / Fact Check, label']
labels_data[cols].head(100)

,source,n_articles,aggregated_label,"Media Bias / Fact Check, label"
0,21stcenturywire,791.0,2.0,conspiracy_pseudoscience
1,abcnews,2861.0,0.0,left_center_bias
2,activistpost,2780.0,2.0,conspiracy_pseudoscience
3,addictinginfo,12.0,1.0,left_bias
4,adobochronicles,417.0,NaN,satire
5,ageofautism,646.0,2.0,conspiracy_pseudoscience
6,airwars,16.0,0.0,least_biased
7,aljazeera,11611.0,0.0,left_center_bias
8,americablognews,104.0,0.0,left_bias
9,anonnews,508.0,NaN,NaN


In [7]:
labels_data.loc[labels_data['source']=='new_york_times'][cols]

,source,n_articles,aggregated_label,"Media Bias / Fact Check, label"


In [4]:
bias_list = ['left_bias', 'left_center_bias', 'right_center_bias', 'right_bias', 'questionable_source']
labels_data = labels_data.loc[labels_data['Media Bias / Fact Check, label'].isin(bias_list)][cols]

In [5]:
# Execute query and load results into pandas dataframe
def execute_query_pandas(path, query):
    conn = sqlite3.connect(path)
    df = pd.read_sql_query(query, conn)
    return df

In [6]:
file = path + 'nela-eng-2019.db'
query= ''' SELECT * FROM newsdata '''
data = execute_query_pandas(file, query)

In [10]:
data = data.loc[data['source'].isin(labels_data['source'])]
for source in labels_data['source']:
    data.loc[data.source == source, 'label'] = labels_data.loc[labels_data.source == source, 'aggregated_label'].values[0]
    data.loc[data.source == source, 'bias'] = labels_data.loc[labels_data.source == source, 'Media Bias / Fact Check, label'].values[0]
cols = ['title', 'content', 'label', 'bias']
data = data[cols].dropna(subset = ['label', 'bias'], how='any')

In [24]:
df['bias'].value_counts()

right_bias             5000
left_bias              5000
right_center_bias      5000
questionable_source    5000
left_center_bias       5000
Name: bias, dtype: int64

In [52]:
samples_per_label = 5000
seed = 0

left_center_bias = data.loc[data['bias'] == 'left_center_bias'].sample(n=samples_per_label, random_state=seed)
right_center_bias = data.loc[data['bias'] == 'right_center_bias'].sample(n=samples_per_label, random_state=seed)
left_bias = data.loc[data['bias'] == 'left_bias'].sample(n=samples_per_label, random_state=seed)
questionable_source = data.loc[data['bias'] == 'questionable_source'].sample(n=samples_per_label, random_state=seed)
right_bias = data.loc[data['bias'] == 'right_bias'].sample(n=samples_per_label, random_state=seed)

In [53]:
df = left_center_bias.append(right_center_bias).append(left_bias).append(questionable_source).append(right_bias)
df_content = df[['content', 'bias']].rename(columns={'content':'text', 'bias':'label'})
df_title = df[['title', 'bias']].rename(columns={'title':'text', 'bias':'label'})

In [54]:
df_content = shuffle(df_content, random_state=seed)
df_title = shuffle(df_title, random_state=seed)

In [55]:
df_content.to_csv(path + 'data_content.csv', header=True, index=False)
df_title.to_csv(path + 'data_title.csv', header=True, index=False)